# Hierarchical Reasoning Model (HRM) Training Notebook

This notebook serves as the entry point for training your HRM model on Kaggle/Colab using their computational resources.

## Setup Instructions:
1. Upload your entire codebase to Google Drive or Kaggle
2. Mount Google Drive (if using Colab)
3. Set the correct project path
4. Install dependencies
5. Run training

In [ ]:
# Mount Google Drive (Colab only)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except ImportError:
    print("Not running in Colab, skipping Drive mount")

# Set your project path
# For Colab with Drive: PROJECT_PATH = '/content/drive/MyDrive/AlgoTrading'
# For Kaggle: PROJECT_PATH = '/kaggle/input/your-dataset/AlgoTrading'
PROJECT_PATH = '/content/drive/MyDrive/AlgoTrading'  # CHANGE THIS TO YOUR ACTUAL PATH

import sys
import os

# Add project root to Python path
sys.path.insert(0, PROJECT_PATH)
sys.path.insert(0, os.path.join(PROJECT_PATH, 'src'))

print(f"Project path added to sys.path: {PROJECT_PATH}")
print(f"Python path: {sys.path[:5]}...")  # Show first 5 paths

In [ ]:
# Install dependencies
!pip install -r "{PROJECT_PATH}/requirements.txt" --quiet
print("Dependencies installed successfully!")

In [ ]:
# Verify imports work correctly
try:
    # Core HRM components
    from src.models.hierarchical_reasoning_model import HierarchicalReasoningModel
    from src.backtesting.environment import TradingEnv
    from src.utils.data_loader import DataLoader
    from src.training.trainer import Trainer
    from src.training.universal_trainer import UniversalTrainer
    
    # Configuration
    from src.utils.config_loader import ConfigLoader
    from src.utils.instrument_loader import load_instruments
    
    # Utilities
    from src.utils.iteration_manager import IterationManager
    from src.utils.research_logger import ResearchLogger
    
    print("✅ All imports successful!")
    print("✅ HRM model components loaded")
    print("✅ Training utilities loaded")
    print("✅ Configuration systems loaded")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please check your project path and file structure")

In [ ]:
# Verify configuration loading
try:
    config_loader = ConfigLoader()
    config = config_loader.get_config()
    
    # Check key configuration sections
    print("✅ Configuration loaded successfully!")
    print(f"Model type: {config.get('model', {}).get('model_type', 'Not specified')}")
    print(f"HRM episodes: {config.get('training_sequence', {}).get('stage_1_hrm', {}).get('episodes', 'Not specified')}")
    print(f"Environment episode length: {config.get('environment', {}).get('episode_length', 'Not specified')}")
    
    # Check instruments
    instruments = load_instruments(os.path.join(PROJECT_PATH, 'config', 'instruments.yaml'))
    print(f"✅ Loaded {len(instruments)} instruments")
    
except Exception as e:
    print(f"❌ Configuration error: {e}")

In [ ]:
# Check data availability
import pandas as pd

DATA_DIR = os.path.join(PROJECT_PATH, 'data', 'final')

if os.path.exists(DATA_DIR):
    try:
        # List available data files
        csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
        print(f"✅ Found {len(csv_files)} CSV files in data directory")
        
        if csv_files:
            print("Sample files:")
            for i, file in enumerate(csv_files[:5]):
                print(f"  {i+1}. {file}")
            
            # Try to load one file to verify format
            sample_file = os.path.join(DATA_DIR, csv_files[0])
            try:
                df = pd.read_csv(sample_file, index_col=0, nrows=5)
                print(f"\n✅ Sample data from {csv_files[0]}:")
                print(f"Shape: {df.shape}")
                print(f"Columns: {list(df.columns)}")
            except Exception as e:
                print(f"⚠️  Could not read sample file: {e}")
        
    except Exception as e:
        print(f"⚠️  Error scanning data directory: {e}")
else:
    print(f"⚠️  Data directory not found: {DATA_DIR}")
    print("Please ensure your data is uploaded to the correct location")

In [ ]:
# Data Processing: Convert Raw Data to Processed Features
print("🔄 Starting data processing pipeline...")

# Use the actual naming convention from the existing system
RAW_DATA_DIR = os.path.join(PROJECT_PATH, 'data', 'raw')
FINAL_DATA_DIR = os.path.join(PROJECT_PATH, 'data', 'final')

# Check for raw data files
raw_files = []
if os.path.exists(RAW_DATA_DIR):
    raw_files = [f for f in os.listdir(RAW_DATA_DIR) if f.endswith('.csv')]
    print(f"📁 Found {len(raw_files)} raw data files")
else:
    print(f"⚠️  Raw data directory not found: {RAW_DATA_DIR}")

# Check for final processed files (actual naming: features_*.csv)
final_files = []
if os.path.exists(FINAL_DATA_DIR):
    final_files = [f for f in os.listdir(FINAL_DATA_DIR) if f.startswith('features_') and f.endswith('.csv')]
    print(f"📁 Found {len(final_files)} processed feature files")
else:
    print(f"📁 Final data directory will be created: {FINAL_DATA_DIR}")

# Determine if we need to run data processing
need_processing = len(final_files) == 0 or len(raw_files) > len(final_files)

if need_processing and len(raw_files) > 0:
    print("🚀 Running feature generation pipeline...")
    
    try:
        # Import and use the actual feature generator used in the system
        from src.data_processing.feature_generator import DynamicFileProcessor
        
        # Initialize the same processor class used in pipeline.py
        processor = DynamicFileProcessor()
        
        print("⚙️  Running feature generation with technical indicators...")
        print("🔧 Processing: OHLCV → Technical Indicators → Market Structure → Features")
        
        # Run the same feature processing logic
        results = processor.process_all_files()
        
        if results:
            print(f"✅ Feature generation completed successfully!")
            
            # Count and verify final files
            final_files_created = [f for f in os.listdir(FINAL_DATA_DIR) if f.startswith('features_') and f.endswith('.csv')]
            print(f"📊 Created {len(final_files_created)} feature files")
            
            # Calculate total rows processed
            total_rows = 0
            for feature_file in final_files_created:
                try:
                    df = pd.read_csv(os.path.join(FINAL_DATA_DIR, feature_file))
                    total_rows += len(df)
                except Exception as e:
                    print(f"⚠️  Could not count rows in {feature_file}: {e}")
            
            print(f"📈 Total rows processed: {total_rows:,}")
            print(f"📁 Final data location: {FINAL_DATA_DIR}")
            
            # Show sample of created files
            print("✅ Created feature files:")
            for i, file in enumerate(final_files_created[:10]):
                print(f"   📄 {file}")
            if len(final_files_created) > 10:
                print(f"   ... and {len(final_files_created) - 10} more files")
                
        else:
            print(f"❌ Feature generation failed: No files processed")
            
    except Exception as e:
        print(f"❌ Data processing pipeline failed: {e}")
        print("⚠️  You may need to manually run the data processing pipeline")
        print("💡 Try running: python src/data_processing/pipeline.py")
        import traceback
        traceback.print_exc()
        
elif len(final_files) > 0:
    print("✅ Processed feature files already exist, skipping data processing")
    print(f"📁 Using existing {len(final_files)} feature files in {FINAL_DATA_DIR}")
    
    # Show sample of existing files
    for i, file in enumerate(final_files[:5]):
        print(f"   📄 {file}")
    if len(final_files) > 5:
        print(f"   ... and {len(final_files) - 5} more files")
        
else:
    print("⚠️  No raw data files found for processing")
    print("💡 Please ensure raw historical data is uploaded to the data/raw directory")
    print("💡 Expected format: CSV files with OHLCV data")

# Read and display sample of processed data
final_files_available = [f for f in os.listdir(FINAL_DATA_DIR) if f.startswith('features_') and f.endswith('.csv')] if os.path.exists(FINAL_DATA_DIR) else []

if final_files_available:
    print("\n" + "="*60)
    print("📊 SAMPLE OF PROCESSED FEATURE DATA")
    print("="*60)
    
    # Read first available processed file
    sample_file = final_files_available[0]
    sample_path = os.path.join(FINAL_DATA_DIR, sample_file)
    
    try:
        df = pd.read_csv(sample_path)
        print(f"📄 Sample from: {sample_file}")
        print(f"📈 Shape: {df.shape} (rows, columns)")
        print(f"🗓️  Date range: {df['datetime_readable'].iloc[0]} to {df['datetime_readable'].iloc[-1]}")
        
        print("\n🔧 Feature Categories:")
        feature_cols = df.columns.tolist()
        print(f"   📊 Price Data: {[col for col in feature_cols if col in ['open', 'high', 'low', 'close']]}")
        print(f"   📈 Moving Averages: {[col for col in feature_cols if 'sma' in col or 'ema' in col][:8]}...")
        print(f"   🎯 Oscillators: {[col for col in feature_cols if any(x in col for x in ['rsi', 'stoch', 'macd', 'cci'])]}")
        print(f"   📏 Volatility: {[col for col in feature_cols if any(x in col for x in ['atr', 'bb_', 'volatility'])]}")
        print(f"   📐 Market Structure: {[col for col in feature_cols if any(x in col for x in ['trend_', 'momentum', 'roc'])]}")
        
        print(f"\n📋 First 3 rows:")
        print(df.head(3).to_string(max_cols=15, max_colwidth=10))
        
        print(f"\n✅ Data contains {len(feature_cols)} technical features ready for HRM training")
        
    except Exception as e:
        print(f"⚠️  Could not read sample data: {e}")

print("\n🔄 Data processing check completed")
print(f"📂 Training will use data from: {FINAL_DATA_DIR}")

In [ ]:
# Set up logging for training
import logging

# Configure clean, minimal logging for training
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Reduce verbosity for specific modules during training
logging.getLogger('src.backtesting.environment').setLevel(logging.WARNING)
logging.getLogger('src.utils.data_loader').setLevel(logging.WARNING)
logging.getLogger('src.data_processing.feature_generator').setLevel(logging.WARNING)
logging.getLogger('src.utils.data_feeding_strategy').setLevel(logging.WARNING)
logging.getLogger('src.utils.config_loader').setLevel(logging.WARNING)
logging.getLogger('src.training.trainer').setLevel(logging.WARNING)
logging.getLogger('src.training.universal_trainer').setLevel(logging.WARNING)

print("✅ Logging configured for training")

In [ ]:
# Main Training Function
def run_hrm_training(episodes=None, testing_mode=False, symbols=None):
    """
    Run HRM training with the specified parameters.
    
    Args:
        episodes (int): Number of episodes to train (overrides config if specified)
        testing_mode (bool): Enable testing mode with reduced episodes
        symbols (list): Specific symbols to train on (None for all available)
    """
    print("🚀 Starting HRM Training")
    
    # Enable detailed logging for direct training runs
    os.environ['DETAILED_BACKTEST_LOGGING'] = 'true'
    
    # Load configuration
    config_loader = ConfigLoader()
    config = config_loader.get_config()
    
    # Determine data directory
    data_processing_config = config.get('data_processing', {})
    if testing_mode:
        data_dir = os.path.join(data_processing_config.get('test_folder', 'data/test'), 'final')
    else:
        data_dir = os.path.join(PROJECT_PATH, 'data/final')
    
    print(f"📂 Using data directory: {data_dir}")
    
    # Initialize data loader
    data_loader = DataLoader(final_data_dir=data_dir, use_parquet=True)
    
    # Determine episodes
    if episodes is not None:
        print(f"📊 Using specified episodes: {episodes}")
    elif testing_mode:
        if 'testing_overrides' in config and 'training_sequence' in config['testing_overrides']:
            episodes = config['testing_overrides']['training_sequence']['stage_1_hrm']['episodes']
        else:
            episodes = 5
        print(f"🧪 Testing mode: {episodes} episodes")
    else:
        if 'training_sequence' in config and 'stage_1_hrm' in config['training_sequence']:
            episodes = config['training_sequence']['stage_1_hrm']['episodes']
        else:
            episodes = 100
        print(f"📊 Production mode: {episodes} episodes")
    
    # Get symbols
    if symbols is not None:
        print(f"🎯 Using specified symbols: {symbols}")
    elif testing_mode:
        symbols = ["RELIANCE_1", "Bank_Nifty_5"]
        print(f"🧪 Testing mode symbols: {symbols}")
    else:
        # Get all available symbols
        symbols = []
        if os.path.exists(data_dir):
            for filename in os.listdir(data_dir):
                if filename.endswith('.csv') and filename.startswith('features_'):
                    symbol = filename.replace('features_', '').replace('.csv', '')
                    symbols.append(symbol)
        symbols = sorted(list(set(symbols)))
        print(f"🔍 Found {len(symbols)} symbols: {symbols[:10]}{'...' if len(symbols) > 10 else ''}")
    
    if not symbols:
        print("❌ No symbols found. Please check your data directory.")
        return
    
    # Set up iteration management
    try:
        iteration_manager = IterationManager(config)
        
        # Setup iteration directory
        iteration_dir = iteration_manager.setup_iteration(data_loader, symbols)
        
        # Initialize research logger
        research_logger = ResearchLogger(config, iteration_dir)
        
        print(f"🔬 Research iteration: {iteration_manager.current_iteration}")
        print(f"📁 Iteration directory: {iteration_dir}")
        
    except Exception as e:
        print(f"⚠️  Iteration management setup failed: {e}")
        iteration_manager = None
        research_logger = None
    
    # Run universal HRM training
    try:
        print("🎯 Starting Universal HRM Training with Symbol Rotation")
        
        # Get configuration sections
        env_config = config.get('environment', {})
        model_config = config.get('model', {})
        
        # Create environment with first symbol to get dimensions
        env = TradingEnv(
            data_loader=data_loader,
            symbol=symbols[0],
            initial_capital=env_config.get('initial_capital', 100000.0),
            lookback_window=env_config.get('lookback_window', 50),
            episode_length=env_config.get('episode_length', 500),
            reward_function=env_config.get('reward_function', "trading_focused"),
            use_streaming=env_config.get('use_streaming', False),
            trailing_stop_percentage=env_config.get('trailing_stop_percentage', 0.02)
        )
        
        # Call reset to initialize observation_space
        env.reset()
        
        # Dynamically get dimensions from the environment
        observation_dim = env.observation_space.shape[0]
        print(f"🔧 Environment configured with Observation Dim: {observation_dim}")
        
        # Update config with the true, environment-derived dimension before creating the model
        config_copy = config.copy()
        config_copy['model'] = config_copy.get('model', {})
        config_copy['model']['observation_dim'] = observation_dim
        
        # Ensure hierarchical_reasoning_model.input_embedding.input_dim is also updated
        hrm_config = config_copy.get('hierarchical_reasoning_model', {})
        input_embedding_config = hrm_config.get('input_embedding', {})
        input_embedding_config['input_dim'] = observation_dim
        hrm_config['input_embedding'] = input_embedding_config
        config_copy['hierarchical_reasoning_model'] = hrm_config
        
        # Create HRM agent
        agent = HierarchicalReasoningModel(config_copy)
        print(f"🤖 HRM Model created with {sum(p.numel() for p in agent.parameters())} parameters")
        
        # Create universal trainer that handles symbol rotation
        trainer = UniversalTrainer(
            agent, symbols, data_loader, 
            num_episodes=episodes, 
            log_interval=10, 
            config=config,
            research_logger=research_logger
        )
        
        print(f"🎯 Training {episodes} episodes with symbol rotation")
        trainer.train()
        
        # Save model
        if not testing_mode:
            model_path = model_config.get('model_path', 'models/universal_final_model.pth')
            model_dir = os.path.dirname(os.path.join(PROJECT_PATH, model_path))
            os.makedirs(model_dir, exist_ok=True)
            
            if hasattr(agent, 'save_model'):
                full_model_path = os.path.join(PROJECT_PATH, model_path)
                agent.save_model(full_model_path)
                print(f"✅ Universal model saved to {full_model_path}")
                
                # Copy model to iteration directory
                if iteration_manager:
                    iteration_manager.save_model_artifacts(full_model_path)
            else:
                print("⚠️  Agent does not have save_model method. Cannot save model")
        else:
            print("🧪 Testing mode - Model not saved")
        
        print("✅ Universal training complete")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# Quick Test Run (Testing Mode)
print("🧪 Running quick test...")
run_hrm_training(episodes=3, testing_mode=True)

In [ ]:
# Full Training Run
# Uncomment the line below to run full training
# run_hrm_training(episodes=2000, testing_mode=False)

In [ ]:
# Custom Training Run with Specific Symbols
# Uncomment the lines below to run training on specific symbols
# symbols_to_train = ["Nifty_5", "Bank_Nifty_15", "RELIANCE_1"]
# run_hrm_training(episodes=500, testing_mode=False, symbols=symbols_to_train)

## Usage Instructions:

1. **Setup**: 
   - Upload your entire AlgoTrading codebase to Google Drive or Kaggle
   - Update `PROJECT_PATH` to point to your codebase location
   - Run the setup cells

2. **Quick Test**:
   - Run the "Quick Test Run" cell to verify everything works
   - This runs 3 episodes in testing mode

3. **Full Training**:
   - Uncomment and run the "Full Training Run" cell
   - Adjust episode count as needed (recommended: 1500-3000 episodes)

4. **Custom Training**:
   - Uncomment and modify the "Custom Training Run" cell
   - Specify symbols and episode count

## Performance Tips:

1. **Use GPU Runtime**: Enable GPU acceleration in Colab/Kaggle
2. **Monitor Progress**: Check logs for training metrics
3. **Save Models**: Models are automatically saved to the models/ directory
4. **Resume Training**: Load saved models for continued training

## Expected Results:

With your 13M+ data points across 55 instruments and 10 timeframes:
- 2000 episodes × 5000 steps = 10M training steps
- Coverage: ~76% of your dataset (with early termination)
- Model Size: FIXED at ~5.6M parameters
- Training Time: 8-16 hours on GPU (much faster than local training)